In [ ]:
# Importando bibliotecas
import datetime as dt
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", None)

In [ ]:
# Armazenando os dados da tabela csv em um DataFrame pandas
folhosas = pd.read_csv('Folhosas.csv')
folhosas.head()

In [ ]:
# Split de dados em dois DataFrames (Mogi e Ibiuna)
folhosas_mogi = folhosas[folhosas['Região'] == "Mogi das Cruzes"]
folhosas_mogi.head()

In [ ]:
# Split de dados em dois DataFrames (Mogi e Ibiuna)
folhosas_ibiuna = folhosas[folhosas['Região'] == "Ibiúna"]
folhosas_ibiuna.head()

In [ ]:
# Armazenando os dados meteorologicos da tabela csv em um DataFrame pandas
inmet_sp = pd.read_csv('INMET\\SP_2013_2023.csv')
inmet_sp.head()

In [ ]:
# Armazenando os dados meteorologicos da tabela csv em um DataFrame pandas
inmet_sorocaba = pd.read_csv('INMET\\SOROCABA_2013_2023.csv')
inmet_sorocaba.head()

In [ ]:
# Limpeza de Outliers
inmet_sp = inmet_sp[(inmet_sp != -9999).all(axis=1)]
inmet_sp.loc[inmet_sp.eq(-9999).any(axis=1)]

In [ ]:
# Limpeza de Outliers
inmet_sorocaba = inmet_sorocaba[(inmet_sorocaba != -9999).all(axis=1)]
inmet_sorocaba.loc[inmet_sorocaba.eq(-9999).any(axis=1)]    

In [ ]:
# Agrupando dados por Data para descobrir a média diária de cada coluna
inmet_sp = inmet_sp.groupby(['Data']).mean()

# Resetando index para que a coluna Data não seja utilizada como tal
inmet_sp.reset_index(inplace=True)

In [ ]:
# Extraindo datas de catálogo de produtos em folhosas mogi
datas_mogi = folhosas_mogi['Data'].unique()

# Encontrando a data anterior a uma semana do primeiro produto
primeira_data = pd.Timestamp(datas_mogi[0]) - dt.timedelta(days=7)

# Adicionando a primeira data as datas descobertas anteriormente
datas_mogi = pd.Series(datas_mogi)
datas_mogi = pd.concat([pd.Series([primeira_data]), datas_mogi], ignore_index=True)

# Convertendo tipo de dados
inmet_sp['Data'] = pd.to_datetime(inmet_sp['Data'])

In [ ]:
# Criando um DataFrame para armazenar os dados do inmet_sp agrupados(+média) pelos intervalos das datas de catálogo em mogi
inmet_folhosas_mogi = pd.DataFrame()

# Foreach para percorrer as datas identificadas e realizar os processamentos de dados
for i in range(len(datas_mogi) - 1):
    intervalo_datas_mogi = inmet_sp[(inmet_sp['Data'] >= datas_mogi[i]) & (inmet_sp['Data'] <= datas_mogi[i+1])]

    pivot_mogi = intervalo_datas_mogi[['Média Precipitação Total (mm)', 'Média Pressão Atmosférica (mB)', 'Média Temperatura do Ar (°C)', 'Média Umidade Relativa do Ar (%)', 'Média Vento Velocidade (m/s)']].mean()
    
    pivot_mogi['Data'] = datas_mogi[i+1]
    
    inmet_folhosas_mogi = inmet_folhosas_mogi.append(pivot_mogi, ignore_index=True)

inmet_folhosas_mogi.head(10)


In [ ]:
# Gerando um novo DataFrame a partir da junção dos dados de produtos e dados meteorológicos históricos de mogi
folhosas_inmet_mogi = pd.merge(folhosas_mogi,inmet_folhosas_mogi)
folhosas_inmet_mogi.head(10)

In [ ]:
# Formatação de colunas (arredondamento de casas decimais)
folhosas_inmet_mogi['Média Precipitação Total (mm)'] = folhosas_inmet_mogi['Média Precipitação Total (mm)'].round(4)
folhosas_inmet_mogi['Média Pressão Atmosférica (mB)'] = folhosas_inmet_mogi['Média Pressão Atmosférica (mB)'].round(3)
folhosas_inmet_mogi['Média Temperatura do Ar (°C)'] = folhosas_inmet_mogi['Média Temperatura do Ar (°C)'].round(2)
folhosas_inmet_mogi['Média Umidade Relativa do Ar (%)'] = folhosas_inmet_mogi['Média Umidade Relativa do Ar (%)'].round(2)
folhosas_inmet_mogi['Média Vento Velocidade (m/s)'] = folhosas_inmet_mogi['Média Vento Velocidade (m/s)'].round(2)

folhosas_inmet_mogi['Média Umidade Relativa do Ar (%)'] = folhosas_inmet_mogi['Média Umidade Relativa do Ar (%)'].astype(str) + '%'

folhosas_inmet_mogi.head()

In [ ]:
inmet = nuppa_inmet.loc[:, 'Data':'Média Vento Velocidade (m/s)']
inmet.to_csv('INMET_SP_2022_2023_filtered.csv', index = 0)